In [ ]:
from PIL import Image  
from PIL import ImageEnhance  
import numpy as np
import os
import random
import csv

#将传入的灰度图二值化
def tow_value(imgry):
    max = 0
    gray = 0
    data =[]
    for i in imgry.getcolors():
        if i[0]>max:
            max = i[0]
            gray = i[1]
        if i[0] < 42:
            data.append(i[1])
    data.append(gray)
    def get_bin_table(data):
        table = []
        for i in range(256):
            if i in data:
                table.append(1)
            else:
                table.append(0)

        return table

    table = get_bin_table(data)
    out = imgry.point(table, '1')
    return out

def depoint(img):
    """传入二值化后的图片进行降噪"""
    pixdata = img.load()
    w,h = img.size
    for y in range(1,h-1):
        for x in range(1,w-1):
            count = 0
            if pixdata[x,y-1] == 0:#上
                count = count + 1
            if pixdata[x,y+1] == 0:#下
                count = count + 1
            if pixdata[x-1,y] == 0:#左
                count = count + 1
            if pixdata[x+1,y] == 0:#右
                count = count + 1
            if pixdata[x-1,y-1] == 0:#左上
                count = count + 1
            if pixdata[x-1,y+1] == 0:#左下
                count = count + 1
            if pixdata[x+1,y-1] == 0:#右上
                count = count + 1
            if pixdata[x+1,y+1] == 0:#右下
                count = count + 1
            if count > 4:
                pixdata[x,y] = 0
    return img

def vectorize(j):
    '''
    将数字字母转换成one-hot向量
    如： 2 => [0,0,1,0,0,0,0,0,0,0...] 长度为62 
    '''
    vector = np.zeros((62, 1))
    if '0' <= j <= '9':
        vector[ord(j)-48] = 1.0
    elif 'A' <= j <= 'Z':
        vector[ord(j)-55] = 1.0
    else:
        vector[ord(j)-61] = 1.0
    return vector

#将切割后的图片信息转换一下
def trans_image(cropped_image):
    a = np.array(cropped_image)
    data = []
    #遍历切割图片的矩阵，转换切割图片的信息
    for i in range(30):
        for j in range(30):
            if a[i][j]:
                data.append(1)
            else:
                data.append(0)
    return np.reshape(data,(900,1))

#加载文件信息并将文件处理成训练模型所需要的数据形式

def load_data(path):
    all_data=[]
    labels = []
    lsdir = os.listdir(path)
    if path == 'train':
        for jpg in lsdir:
            image = Image.open(r'train/'+jpg)
            imgry = image.convert('L')  # 转化为灰度图
            out = tow_value(imgry) #将灰度图二值化
            im =  depoint(out) #将二值化后的图片降噪
            #将图片切割
            cropped_image_1 = im.crop((0, 0, 30, 30))
            cropped_image_2 = im.crop((30, 0, 60, 30))
            cropped_image_3 = im.crop((60, 0, 90, 30))
            cropped_image_4 = im.crop((90, 0, 120, 30))
            cropped_image_5 = im.crop((120, 0, 150, 30))
            all_data.append(trans_image(cropped_image_1))
            all_data.append(trans_image(cropped_image_2))
            all_data.append(trans_image(cropped_image_3))
            all_data.append(trans_image(cropped_image_4))
            all_data.append(trans_image(cropped_image_5))
            labels.append(vectorize(jpg[0]))
            labels.append(vectorize(jpg[1]))
            labels.append(vectorize(jpg[2]))
            labels.append(vectorize(jpg[3]))
            labels.append(vectorize(jpg[4]))
        return list(zip(all_data,labels))
    else:
        for jpg in range(20000):
            image = Image.open(r'test/'+str(jpg)+'.jpg')
            imgry = image.convert('L')  # 转化为灰度图
            out = tow_value(imgry) #将灰度图二值化
            im =  depoint(out) #将二值化后的图片降噪
              #将图片切割
            cropped_image_1 = im.crop((0, 0, 30, 30))
            cropped_image_2 = im.crop((30, 0, 60, 30))
            cropped_image_3 = im.crop((60, 0, 90, 30))
            cropped_image_4 = im.crop((90, 0, 120, 30))
            cropped_image_5 = im.crop((120, 0, 150, 30))
            all_data.append(trans_image(cropped_image_1))
            all_data.append(trans_image(cropped_image_2))
            all_data.append(trans_image(cropped_image_3))
            all_data.append(trans_image(cropped_image_4))
            all_data.append(trans_image(cropped_image_5))
        return all_data
           
            
train_data = load_data('train')
test_data = load_data('test')
print('执行完毕')

class Network(object):

    def __init__(self, sizes):
        """sizes包含各层神经元的数目，比如如果sizes=[2,3,1]说明此神经网络
        共有三层，每一层的神经元数目分别为2，3，1.权重和偏置根据标准正态分
        布随机初始化，第一层为输入神经元，没有偏置"""
        self.num_layers = len(sizes)#层数
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]#偏置，随机初始化，标准正态分布
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]#权重，随机初始化，标准正态分布

    def feedforward(self, a):
        """返回结果"""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """使用小批量随机训练神经网络
         梯度下降。 ``training_data``是一个元组列表
         ``（x，y）``代表训练输入和期望的
        输出。 其他非可选参数是
        不言自明的。 如果提供了``test_data``则
         每次之后，都会根据测试数据评估网络
         迭代，并打印出部分进度。 这对于
         跟踪进度，但会大大降低进度"""
        training_data = list(training_data)
#         test_data = list(test_data)
        if test_data: n_test = len(test_data)#如果提供测测试集，每个epoch后进行评估
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)#每个epoch随机打乱训练集，保证随机性
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))#随时监控程序运行情况，可以删掉
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """更新权重和偏置，eta表示学习率"""
        nabla_b = [np.zeros(b.shape) for b in self.biases]#初始化偏置
        nabla_w = [np.zeros(w.shape) for w in self.weights]#初始化权重
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """反向传播算法，返回梯度，先不要管."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """评估预测正确的个数
            np.argmax函数返回数组的最大值的序号，实现从one-hot到数字的转换；
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """反向传播中用到的计算偏导数，先不要管"""
        return (output_activations-y)

    def predict(self,test_data):
        '''预测函数'''
        predict = []
        print(self.biases)
        print(self.weights)
        for x in test_data:
            p = np.argmax(self.feedforward(x))
            if 0<= p <= 9:
                predict.append(str(p))
            elif 10 <= p <= 35:
                predict.append(chr(p+55))
            else:
                predict.append(chr(p+61))
        return predict

def sigmoid(z):
    """S型函数"""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """反向传播中用到的."""
    return sigmoid(z)*(1-sigmoid(z))

net = Network([900,100,62])
net.SGD(train_data,30,500,2.0)#30个迭代期，小批量为150，学习率3.0
print('训练完毕')

#生成预测结果
pre = net.predict(test_data)
predict = []
for i in range(0,100000,5):
    predict.append(pre[i]+pre[i+1]+pre[i+2]+pre[i+3]+pre[i+4])

#将预测结果写入csv文件
index = range(0,predict.__len__()+1)
headers = ['id','y']
rows = list(zip(index,predict))
with open('predict.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)